In [1]:
# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)
    faces = []
    locs = []
    preds = []
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            faces.append(face)
            locs.append((startX, startY, endX, endY))
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)
    return (locs, preds)

prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)
maskNet = load_model("models/drowsiness_detector.h5")
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()

# loop over the frames from the video stream
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=224)
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        yawn_detection_dict = ('Closed','Open','no_yawn','yawn')
        res = yawn_detection_dict[np.argmax(pred,axis=0)]
        color = ()
        
        if res == 'Open':
            color = (0,255,0)
        elif res == 'Closed':
            color = (0,0,255)
        elif res == 'yawn':
            color = (255,0,255) # purple
        else:
            color = (0,153,153)

        cv2.putText(frame, res, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()


[INFO] starting video stream...
(1, 1, 124, 7)
1/1 [==============================] - 0s 426ms/step
(1, 1, 137, 7)
1/1 [==============================] - 0s 34ms/step
(1, 1, 131, 7)
1/1 [==============================] - 0s 36ms/step
(1, 1, 133, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 129, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 126, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 138, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 130, 7)
1/1 [==============================] - 0s 32ms/step
(1, 1, 123, 7)
1/1 [==============================] - 0s 23ms/step
(1, 1, 115, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 124, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 123, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 120, 7)
1/1 [==============================] - 0s 32ms/step
(1, 1, 119, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 125, 7)
1/1 [=========

1/1 [==============================] - 0s 26ms/step
(1, 1, 119, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 132, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 120, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 122, 7)
1/1 [==============================] - 0s 23ms/step
(1, 1, 127, 7)
1/1 [==============================] - 0s 22ms/step
(1, 1, 127, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 126, 7)
1/1 [==============================] - 0s 23ms/step
(1, 1, 120, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 123, 7)
1/1 [==============================] - 0s 22ms/step
(1, 1, 124, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 126, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 127, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 120, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 125, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 126

1/1 [==============================] - 0s 24ms/step
(1, 1, 128, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 135, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 131, 7)
1/1 [==============================] - 0s 23ms/step
(1, 1, 131, 7)
1/1 [==============================] - 0s 23ms/step
(1, 1, 132, 7)
1/1 [==============================] - 0s 22ms/step
(1, 1, 128, 7)
1/1 [==============================] - 0s 23ms/step
(1, 1, 129, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 127, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 138, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 131, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 131, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 125, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 125, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 128, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 130

1/1 [==============================] - 0s 27ms/step
(1, 1, 131, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 126, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 126, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 119, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 123, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 121, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 123, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 129, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 129, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 135, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 127, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 125, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 118, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 120, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 118

1/1 [==============================] - 0s 28ms/step
(1, 1, 128, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 136, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 124, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 127, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 128, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 137, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 129, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 129, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 127, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 127, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 126, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 133, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 130, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 128, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 128

...

1/1 [==============================] - 0s 27ms/step
(1, 1, 135, 7)
1/1 [==============================] - 0s 29ms/step
(1, 1, 131, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 131, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 125, 7)
1/1 [==============================] - 0s 32ms/step
(1, 1, 130, 7)
1/1 [==============================] - 0s 32ms/step
(1, 1, 129, 7)
1/1 [==============================] - 0s 32ms/step
(1, 1, 130, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 129, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 133, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 128, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 128, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 130, 7)
1/1 [==============================] - 0s 33ms/step
(1, 1, 129, 7)
1/1 [==============================] - 0s 32ms/step
(1, 1, 130, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 134

1/1 [==============================] - 0s 25ms/step
(1, 1, 125, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 119, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 122, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 110, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 112, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 111, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 117, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 116, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 113, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 113, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 117, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 116, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 111, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 113, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 117

1/1 [==============================] - 0s 27ms/step
(1, 1, 135, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 133, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 135, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 130, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 133, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 135, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 140, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 120, 7)
1/1 [==============================] - 0s 29ms/step
(1, 1, 129, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 137, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 118, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 124, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 116, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 125, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 129

1/1 [==============================] - 0s 27ms/step
(1, 1, 137, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 142, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 141, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 135, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 137, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 140, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 139, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 137, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 133, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 135, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 130, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 140, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 144, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 145, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 142

1/1 [==============================] - 0s 27ms/step
(1, 1, 114, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 114, 7)
1/1 [==============================] - 0s 28ms/step
(1, 1, 118, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 124, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 127, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 125, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 137, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 139, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 136, 7)
1/1 [==============================] - 0s 31ms/step
(1, 1, 128, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 128, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 134, 7)
1/1 [==============================] - 0s 25ms/step
(1, 1, 135, 7)
1/1 [==============================] - 0s 26ms/step
(1, 1, 124, 7)
1/1 [==============================] - 0s 27ms/step
(1, 1, 133